In [2]:
!pip install tensorflow
!pip install googledrivedownloader
!pip install plotly

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy\\compat\\py3k.py'
Consider using the `--user` option or check the permissions.




  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=33f3b45381f8ee66025005e9c7f779674228f77f5b01c1c0c0542daa976573e5
  Stored in directory: c:\users\hem\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:


# Introduction


Note: This notebook uses `python 3` and these packages: `tensorflow`, `pandas`, `matplotlib`, `scikit-learn`.

## Importing Libraries & Helper Functions

First of all, we will need to import some libraries and helper functions. This includes TensorFlow and some utility functions that I've written to save time.

In [ ]:
#IMPORT THE NECESSARY LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import seaborn as sns

import matplotlib.ticker as mtick
plt.style.use('fivethirtyeight')
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import  ExtraTreesRegressor
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from utils import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

In [ ]:
#!ls /content/

#  Importing the Data

The dataset is saved in a `base_merged_2.csv` file. We will use `pandas` to take a look at some of the rows.

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
file_name = './base_merged_2.csv'
gdd.download_file_from_google_drive(file_id='1rYw4OQ-uK3YJohQmekV-Drio2wlEHA9N',
                                    dest_path=file_name,
                                    unzip=False)

In [ ]:
data.dtypes #checking the data types

In [ ]:
data.isna().sum() 

In [ ]:
#Deleting Unnnecessary Columns
df=data.drop(['MTW Status','TaxPaidperReturn','TaxPaidperReturn','IncomeperReturn'],axis=1) #Dropping the column like "phone" and "url" and saving the new dataset as "df"

In [ ]:
#You can use pandas profiling to get an over all overview of the dataset
import pandas_profiling as pf

pf.ProfileReport(df)

**Checking for duplicate values**

In [ ]:
df.duplicated().sum() 

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

**Drop null values**

In [ ]:
#Remove the NaN values from the dataset
df.dropna(how='any',inplace=True)
df.isnull().sum()

**Renaming columes appropriately**

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'Price_Rng':'Price_Range',
                        'SalaryperReturn':'User_salary', 
                        'Ratings':'Review_rating' , 
                        'Reviews':'Total_reviews',
                        'NumberReturns':'Population', 
                        'Zip-Code':'Zip_code', 
                        'AdjustedGross Income':'Gross_income'})
df.columns

In [ ]:
df.Cuisine.unique()

In [ ]:
df.head()

**Cleaning the dataset**

In [ ]:
df['User_salary'].unique()

**replacing the "," and "$" with nothing and converting the results to float**

In [ ]:
df['User_salary'] = df['User_salary'].apply(lambda x: x.replace('$','')) #Using lambda function to replace '$' from User_salary
df['User_salary'] = df['User_salary'].apply(lambda x: x.replace(',','')) #Using lambda function to replace ',' from User_salary
df['User_salary'] = df['User_salary'].astype(float)

In [ ]:
df['Population'] = df['Population'].apply(lambda x: x.replace(',','')) #Using lambda function to replace ',' from Population
df['Population'] = df['Population'].astype(float)

In [ ]:
df['Gross_income'] = df['Gross_income'].apply(lambda x: x.replace(',','')) #Using lambda function to replace ',' from Gross_income
df['Gross_income'] = df['Gross_income'].astype(float)

In [ ]:
df['Name'] = df['Name'].apply(lambda x: x.replace("'",'')) #Using lambda function to replace ',' from Gross_income

In [ ]:
print(df['User_salary'].unique())
print('---'*10)
df.dtypes

In [ ]:
df['Review_rating'].unique()

# Visualisations

In [ ]:
plt.figure(figsize=(17,10))
chains=df['Name'].value_counts()[:20]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most famous restaurants")
plt.xlabel("Number of outlets")
plt.show()

## **Rating Distributions**

In [ ]:
plt.figure(figsize=(9,7))
sns.distplot(df['Review_rating'],bins=20)

*  **Insight**

  We can infer from above that most of the ratings are within 3.5 and 4.5



In [ ]:
#Distribution of the cost Vs ratings in parallel with online order
plt.figure(figsize=(5,4))
sns.scatterplot(x="Review_rating",y='Total_reviews',hue='Price_Range',data=df)
plt.show() 

In [ ]:
#Distribution of the cost Vs ratings in parallel with online order
plt.figure(figsize=(10,25))
sns.scatterplot(x="Price_Range",y='Cuisine',hue='Review_rating',data=df)
plt.show() 

## **Cuisines Types**

In [ ]:
#Types of Cuisine

sns.countplot(df['Cuisine']).set_xticklabels(sns.countplot(df['Cuisine']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(20,12)
plt.title('Cuisine')

In [ ]:
trace0=go.Box(y=df['Price_Range'],name="Accepting Price_Range",
              marker = dict(
        color = 'rgb(113, 10, 100)',
    ))
data=[trace0]
layout=go.Layout(title="Box plot of Approximate Price Range",width=800,height=800,yaxis=dict(title="Price_Range"))
fig=go.Figure(data=data,layout=layout)
py.iplot(fig)

In [ ]:
plt.figure(figsize=(8,8))
sns.distplot(df['Price_Range'])
plt.show()

In [ ]:
sns.heatmap(df.corr(),annot = True) #Just to see the correlation between the features and the label

# **Building Our Model**

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


# Name              object
# Cuisine           object
# Review_rating    float64
# Total_reviews      int64
# Price_Range        int64
# Zip_code           int64
# State             object
# City              object
# Population       float64
# Gross_income     float64
# User_salary      float64

In [ ]:
df.dtypes

In [ ]:
# Correlation matrix for all characteristics
corrMat = df.corr()

# Determine which correlations are significant, and drop the others
# Assuming correlation values >= 0.8 are significant
corrValues = corrMat.unstack().abs()
c = [corrValues.drop(i, inplace = True) for i, v in corrValues.items()
     if i[0] == i[1]                           # left and right index are the same
     or v < 0.8                                # Value is insignificant
     or (i[1], i[0]) in corrValues.index]      # repeated correlation values

# Sort and print the correlation values
print('Characteristics with significant correlations:\n{}\n'.format(corrValues.sort_values(ascending = False)))

# Print the figure
sns.set(font_scale = 1.0, rc = {'figure.figsize': (12, 10)})
sns.heatmap(corrMat, vmax = 0.8, square = True)

In [ ]:
# df = df.iloc[:,1:] 
# df_norm = (df - df.mean()) / df.std() #Data is normalized by subtracting each value in the column with the mean value and then dividing it with the standard                                                 deviation of the whole column
# df_norm.head()

In [ ]:
# Name              object
# Cuisine            int32
# Review_rating    float64
# Total_reviews      int64
# Price_Range        int64
# Zip_code           int64
# State             object
# City              object
# Population       float64
# Gross_income     float64
# User_salary      float64
df.head()

In [ ]:
df.Cuisine = le.fit_transform(df.Cuisine) # direct number convert 


# Converting the Cuisine into numerical_values using get_dummies method
dummy_cols = pd.get_dummies(df.Cuisine)
df_1 = pd.concat([df,dummy_cols], axis='columns')

In [ ]:
df_1.head()

In [ ]:
# own_data = df.iloc[:,[2,3,4,5,6,9,10,11]]
own_data = df_1.drop([ "User_salary","Name","Cuisine","State","City","id"], axis='columns')
own_data.to_csv('own_data.csv') 
own_data.head()

In [ ]:
df.head()

In [ ]:
df = df.drop(["Gross_income" ,"Name","Total_reviews", "State","City","id"], axis='columns')

In [ ]:
df.head()

In [ ]:
# X1 = df.drop(["Gross_income" ,'User_salary',"Name","Total_reviews", "Cuisine","State","City","id"], axis='columns')
# y1 = df['User_salary']

In [ ]:
# X = df_1.drop(["Gross_income" ,'User_salary',"Name","Total_reviews", "Cuisine","State","City","id"], axis='columns')
# y = df_1['User_salary']

# Data Normalization

We can make it easier for optimization algorithms to find minimas by normalizing the data before training a model.

In [ ]:

df_norm = (df - df.mean()) / df.std() #Data is normalized by subtracting each value in the column with the mean value and then dividing it with the standard                                                 deviation of the whole column
df_norm.head()


In [1]:
# print(convert_label_value(_______), " that corresponds to the User_salary _______")

# Create Training and Test Sets

## Select Features

Make sure to remove the column User_salary from the list of features as it is the label and should not be used as a feature.

In [ ]:
X = df_norm.iloc[:, :5] #Storing the features in 'X'
X.head()

##  Select Labels
We select the User_salary

In [ ]:
Y = df_norm.iloc[:, -1] #Storing the labels in 'Y'
Y.head()

##  Feature and Label Values

We will need to extract just the numeric values for the features and labels as the TensorFlow model will expect just numeric values as input.

In [ ]:
X_arr = X.values
Y_arr = Y.values

print('X_arr shape: ', X_arr.shape) #'shape' gives the dimension of the entity
print('Y_arr shape: ', Y_arr.shape)

## Train and Test Split

We will keep some part of the data aside as a __test__ set. The model will not use this set during training and it will be used only for checking the performance of the model in trained and un-trained states. This way, we can make sure that we are going in the right direction with our model training.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, Y_arr, test_size = 0.05, shuffle = True, random_state=0) 
#This predefined function splits the dataset to train and test set, where test size is given in 'test_size'(Here 5%) 
#Random state ensures that the splits that you generate are reproducible. Scikit-learn uses random permutations to generate the splits.

print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

##  Convert Label Value

Because we are using normalized values for the labels, we will get the predictions back from a trained model in the same distribution. So, we need to convert the predicted values back to the original distribution if we want predicted User_salares.

In [ ]:
y_mean = df['User_salary'].mean()
y_std = df['User_salary'].std()

def convert_label_value(pred):       #Defining a function which will convert the label values back to the original distribution and return it
    return int(pred * y_std + y_mean)

#  Create the Model

A function that returns an untrained model of a certain architecture. We're using a simple neural network architecture with just three hidden layers. We're going to use the relu activation function on all
the layers except for the output layer.


We know that input shape is simply a list of 5 values because we just have 5 features. 

The activation is going to be really or rectified linear unit, and the next layer will be again a fully connected or dense
layer. 

And this time, let's use 20 nodes and again,  the same activation function relu and one more hidden layer with 5.
nodes and finally, the output layer with just 1 node. so we have essentially, we have three hidden
layers.

In [ ]:
## Create the Model

Let's write a function that returns an untrained model of a certain architecture.

In [ ]:
from utils import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def get_model():
    
    model = Sequential([
        Dense(10, input_shape = (5,), activation = 'relu'), #10 neurons, Input Layer
        Dense(20, activation = 'relu'),                     #20 neurons, Hidden Layer
        Dense(5, activation = 'relu'),                      #5  neurons, Hidden Layer
        Dense(1)                                            #Output Layer
    ])                                                      #'relu' activation

    model.compile(
        loss='mse',                                         #Trained using Mean square error loss (Cost function) 
        optimizer='adam',                                    #Optimizer used is 'adam' (One of the Fastest optimizers)
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )
    
#     model.compile(
#         optimizer='sgd',
#         loss='binary_crossentropy',
#         metrics=['accuracy'])
    
    
    return model

model = get_model()
model.summary()

This is the input and then we have three hidden layers.
but 10, 20 and five nodes respectively.
All the layers have activation function set to value
except for the output layer.
Since this is a regression problem, we just need the linear
output without any activation function here.

The first dense layer that you see here is our first
hidden there, which has 10 nodes.

The next one has 20 nodes.
Next one has 5 nodes.
And the final one node, the output there has one, and you can see
that we have trainable parameters count 401.

Because these are dense layers, they are fully
connected layers and if you want to understand how
these parameters,count is arrived at, you can simply
multiply the nodes in your output layer in in any
one of these layers with the notes in the proceeding here.

So if you if you take a look at dense to for example,
we have 5 nodes and in the preceding layer that
is connected to we have 20 nodes, so each no disconnected.


Each node of dense one is connected to each note of dense two,
which means we have total 100 connections.
But you see that you have 105  parameters
for the Slayer.
Why is that?
That's simply because even though we have 100 weights, we
also have a bias or intercept,  in every layer.

Now that is just one interceptor connected to all the nodes
of the layer that you calculating these parameters for so
520 gives you 100 and five into one gives you 5,
and the total is 105 and you can do this exercise for all
the layers and arrive at the same number of trainable
parameters.



In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience = 100) #Defining early stopping parameter (optional, to save time)

# model = get_model()

# preds_on_untrained = model.predict(X_test) #Make predictions on the test set before training the parameters

# #Finally training the model-->
# history = model.fit(
#     X_train, y_train,
#     validation_data = (X_test, y_test),
#     epochs = 1000,
#     callbacks = [early_stopping]
# )

#  Model Training



We can use an `EarlyStopping` callback from Keras to stop the model training if the validation loss stops decreasing for a few epochs.

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience = 10) #Defining early stopping parameter (optional, to save time)

model = get_model()

preds_on_untrained = model.predict(X_test) #Make predictions on the test set before training the parameters

#Finally training the model-->
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = 6,
    callbacks = [early_stopping]
)

##  Plot Training and Validation Loss

Let's use the `plot_loss` helper function to take a look training and validation loss.

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
The training and validation loss.
Values decreased as the training then don, and that's great.
But we don't know yet if the train model actually makes
reasonably accurate predictions.
So let's take a look at that.
Now. Remember that we had some predictions on the untrained
model. Similarly, we will make some predictions on the train
model on the same data set, of course, which is X test.

In [ ]:
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

In [ ]:
loss = model.evaluate(X_test, y_test)
print("Loss is : ",loss)

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
plot_history(history)

#  Predictions

In [ ]:
##  Plot Raw Predictions

Let's use the `compare_predictions` helper function to compare predictions from the model when it was untrained and when it was trained.

In [ ]:
def plot_predictions(preds, y_test):
    plt.figure(figsize=(8, 8))
    plt.plot(preds, y_test, 'ro')
    plt.xlabel('Preds')
    plt.ylabel('Labels')
    plt.xlim([-0.5, 0.5])
    plt.ylim([-0.5, 0.5])
    plt.plot([-0.5, 0.5], [-0.5, 0.5], 'b--')
    plt.show()
    return

In [ ]:
history.history

In [ ]:
def compare_predictions(preds1, preds2, y_test):
    plt.figure(figsize=(8, 8))
    plt.plot(preds1, y_test, 'ro', label='Untrained Model')
    plt.plot(preds2, y_test, 'go', label='Trained Model')
    plt.xlabel('Preds')
    plt.ylabel('Labels')
    
    y_min = min(min(y_test), min(preds1), min(preds2))
    y_max = max(max(y_test), max(preds1), max(preds2))
    
    plt.xlim([y_min, y_max])
    plt.ylim([y_min, y_max])
    plt.plot([y_min, y_max], [y_min, y_max], 'b--')
    plt.legend()
    plt.show()
    return

##  Plot Raw Predictions

Let's use the `compare_predictions` helper function to compare predictions from the model when it was untrained and when it was trained.

In [ ]:
preds_on_trained = model.predict(X_test)
compare_predictions(preds_on_untrained, preds_on_trained, y_test)

In [ ]:
## Plot User salary Predictions

The plot for User_salary predictions and raw predictions will look the same with just one difference: The x and y axis scale is changed.

In [ ]:
User_salary_on_untrained = [convert_label_value(y) for y in preds_on_untrained]
User_salary_on_trained = [convert_label_value(y) for y in preds_on_trained]
User_salary_y_test = [convert_label_value(y) for y in y_test]

compare_predictions(User_salary_on_untrained, User_salary_on_trained, User_salary_y_test)

In [ ]:
We pretty much get the same graph, but the ranges now different. You can see the ranges  or something for both predictions and labels 
You can see that the train model is a lot more same as in its predictions to ground truth compared to the untrained model.

In [ ]:
# import pickle 
# # Saving model to disk
# pickle.dump(ET_Model, open('dl_model.pkl','wb'))
# model=pickle.load(open('model.pkl','rb'))